#Pruebas modelos (Only PVPC)

##Activación para Colab

In [0]:
#LMG: Variable para controlar si estamos en Colab o no y aplicar a las celdas:
inColab = True

En la siguiente celda, montamos la carpeta personal del Drive en Colab en caso de estar en este entorno. Pedirá un código al que se accede desde el enlace que facilita.

In [2]:
#LMG: Para Google CoLab tener el repo de Drive:
if inColab:
  from google.colab import drive
  drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##Carga de librerías y preparación de datos

Antes de preparar todo, elegimos el modelo a utilizar:

In [0]:
useDense = False
useLSTM = True

Ahora pasamos a la carga de librerías y preparación de datos:

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout, SimpleRNN, BatchNormalization
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from statsmodels.graphics.tsaplots import plot_acf

Using TensorFlow backend.


In [0]:
#LMG: Para CoLab en sample_data:
if inColab:
  data_consumo = pd.read_csv("/content/drive/My Drive/TFM/01.Utils/data/data_total (1).csv")
else:
  data_consumo = pd.read_csv("data_total.csv")
  
data_consumo.head()

,Unnamed: 0,Unnamed: 0.1,fecha,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,date_day,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Precio mercado SPOT Diario_x,Demanda real,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia,Precio SPOT PT,Precio SPOT FR,PVPC-target
0,0,0,2015-10-01 00:00:00+02:00,117.77,69.49,71.88,1.443650e+09,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,2015-10-01,47.48,0.0,61.18,NaN,56.65,25305.0,267.6,182.4,-606.6,NaN,NaN,1200.0,56.65,38.56,123.30
1,1,1,2015-10-01 01:00:00+02:00,117.09,68.21,65.57,1.443654e+09,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,2015-10-01,47.48,0.0,59.59,NaN,53.50,23959.0,195.7,-269.5,-1112.5,NaN,NaN,1200.0,53.50,31.27,121.98
2,2,2,2015-10-01 02:00:00+02:00,114.59,65.34,62.53,1.443658e+09,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,2015-10-01,47.48,0.0,56.27,49.13,49.69,22873.0,202.1,94.0,-951.0,NaN,NaN,1200.0,49.69,30.66,126.70
3,3,3,2015-10-01 03:00:00+02:00,116.60,66.53,63.40,1.443661e+09,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,2015-10-01,47.48,0.0,45.99,45.50,49.10,22550.0,61.3,319.7,-846.3,NaN,NaN,1200.0,49.10,29.14,131.63
4,4,4,2015-10-01 04:00:00+02:00,122.07,71.03,67.37,1.443665e+09,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,2015-10-01,47.48,0.0,52.28,49.13,51.25,22017.0,45.1,405.5,-760.5,NaN,NaN,1200.0,51.25,29.40,134.73


In [0]:
data = data_consumo[['PVPC_DEF']]

data.head()

,PVPC_DEF
0,117.77
1,117.09
2,114.59
3,116.60
4,122.07


###Formateo de datos para los modelos

Generamos el dataset como secuencias de precio:

In [0]:
# split a sequence into samples
def split_sequence(sequence, n_steps):
  X = list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x = sequence[i:end_ix,:]
    X.append(seq_x)

  return np.array(X)

In [0]:
split_steps = 100

data_processed = split_sequence(data.to_numpy()[:,0:1], split_steps)
x_data = data_processed[:,:-1,0]
y_data = data_processed[:,-1]
# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.33, shuffle=False)

#Convert NaN values to 0:
x_train = np.nan_to_num(x_train)
x_valid = np.nan_to_num(x_valid)
y_train = np.nan_to_num(y_train)
y_valid = np.nan_to_num(y_valid)

print('Xtrain_dim:', x_train.shape)
print('Ytrain_dim:', y_train.shape)

Xtrain_dim: (20404, 99)
Ytrain_dim: (20404, 1)


Veamos cómo queda:

In [0]:
x_train_DF = pd.DataFrame(x_train)
x_train_DF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98
0,0.68,2.50,-2.01,-5.47,-3.24,-5.39,-4.99,4.67,0.51,1.77,1.78,2.10,5.61,5.36,-6.72,-10.85,-4.84,4.64,11.70,7.24,-8.35,-1.53,1.56,-2.26,1.32,-4.72,-4.93,-3.10,2.54,-0.02,1.67,2.42,0.59,2.61,5.37,7.42,4.08,-4.67,-8.25,-8.26,...,1.45,-1.55,-10.01,-12.93,-7.22,8.30,9.59,8.67,3.80,0.76,1.39,0.00,1.73,0.49,-2.69,-3.39,-4.40,0.40,4.86,-0.70,1.75,4.22,5.62,3.69,2.71,-1.29,-11.62,-11.47,-13.30,3.53,14.98,7.55,4.03,0.79,-0.43,-2.30,0.08,-1.05,-10.81,-6.86
1,2.50,-2.01,-5.47,-3.24,-5.39,-4.99,4.67,0.51,1.77,1.78,2.10,5.61,5.36,-6.72,-10.85,-4.84,4.64,11.70,7.24,-8.35,-1.53,1.56,-2.26,1.32,-4.72,-4.93,-3.10,2.54,-0.02,1.67,2.42,0.59,2.61,5.37,7.42,4.08,-4.67,-8.25,-8.26,-7.55,...,-1.55,-10.01,-12.93,-7.22,8.30,9.59,8.67,3.80,0.76,1.39,0.00,1.73,0.49,-2.69,-3.39,-4.40,0.40,4.86,-0.70,1.75,4.22,5.62,3.69,2.71,-1.29,-11.62,-11.47,-13.30,3.53,14.98,7.55,4.03,0.79,-0.43,-2.30,0.08,-1.05,-10.81,-6.86,1.69
2,-2.01,-5.47,-3.24,-5.39,-4.99,4.67,0.51,1.77,1.78,2.10,5.61,5.36,-6.72,-10.85,-4.84,4.64,11.70,7.24,-8.35,-1.53,1.56,-2.26,1.32,-4.72,-4.93,-3.10,2.54,-0.02,1.67,2.42,0.59,2.61,5.37,7.42,4.08,-4.67,-8.25,-8.26,-7.55,14.92,...,-10.01,-12.93,-7.22,8.30,9.59,8.67,3.80,0.76,1.39,0.00,1.73,0.49,-2.69,-3.39,-4.40,0.40,4.86,-0.70,1.75,4.22,5.62,3.69,2.71,-1.29,-11.62,-11.47,-13.30,3.53,14.98,7.55,4.03,0.79,-0.43,-2.30,0.08,-1.05,-10.81,-6.86,1.69,-4.61
3,-5.47,-3.24,-5.39,-4.99,4.67,0.51,1.77,1.78,2.10,5.61,5.36,-6.72,-10.85,-4.84,4.64,11.70,7.24,-8.35,-1.53,1.56,-2.26,1.32,-4.72,-4.93,-3.10,2.54,-0.02,1.67,2.42,0.59,2.61,5.37,7.42,4.08,-4.67,-8.25,-8.26,-7.55,14.92,9.09,...,-12.93,-7.22,8.30,9.59,8.67,3.80,0.76,1.39,0.00,1.73,0.49,-2.69,-3.39,-4.40,0.40,4.86,-0.70,1.75,4.22,5.62,3.69,2.71,-1.29,-11.62,-11.47,-13.30,3.53,14.98,7.55,4.03,0.79,-0.43,-2.30,0.08,-1.05,-10.81,-6.86,1.69,-4.61,0.78
4,-3.24,-5.39,-4.99,4.67,0.51,1.77,1.78,2.10,5.61,5.36,-6.72,-10.85,-4.84,4.64,11.70,7.24,-8.35,-1.53,1.56,-2.26,1.32,-4.72,-4.93,-3.10,2.54,-0.02,1.67,2.42,0.59,2.61,5.37,7.42,4.08,-4.67,-8.25,-8.26,-7.55,14.92,9.09,4.25,...,-7.22,8.30,9.59,8.67,3.80,0.76,1.39,0.00,1.73,0.49,-2.69,-3.39,-4.40,0.40,4.86,-0.70,1.75,4.22,5.62,3.69,2.71,-1.29,-11.62,-11.47,-13.30,3.53,14.98,7.55,4.03,0.79,-0.43,-2.30,0.08,-1.05,-10.81,-6.86,1.69,-4.61,0.78,2.12


In [0]:
y_train_DF = pd.DataFrame(y_train)
y_train_DF.head()

,0
0,1.69
1,-4.61
2,0.78
3,2.12
4,3.17


Por último, generamos funciones y variables de utilidad a lo largo de todos los modelos:

In [0]:
def rmse (y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred -y_true), axis=-1))


## ????
def mad (y_true, y_pred): 
  return K.sqrt(K.mean(K.square(y_pred -y_true), axis=-1))

##Ejemplo de NN

Red neuronal densa como punto de partida.

###Preparación y entrenamiento

Primero, estandarizamos los **datos de entrenamiento**:

In [0]:
if useDense:
  scaler = MinMaxScaler()
  x_train_data = scaler.fit_transform(x_train)
  y_train_data = scaler.fit_transform(y_train)
  x_valid_data = x_valid
  y_valid_data = y_valid

  print('Entrenamiento:',x_train_data.shape, y_train_data.shape)
  print('Test:',x_valid_data.shape, y_valid_data.shape)

Por último, definimos los tamaños de nuestra red y algunas variables adicionales

In [0]:
if useDense:
  batch_size = 5
  num_var = x_train_data.shape[1]
  hidden_size = 20
  output_size = y_train_data.shape[1]

###Modelo

In [0]:
if useDense:
  # define model
  model = Sequential()
  model.add(Dense(hidden_size, 
                  activation='relu',
                  input_shape=(num_var,),
                  kernel_regularizer=l2(1e-2)))
  model.add(Dropout(0.2))
  model.add(Dense(hidden_size, 
                  activation='relu',
                  kernel_regularizer=l2(1e-2)))
  model.add(Dropout(0.2))
  model.add(Dense(output_size,
                  activation='linear',
                  kernel_regularizer=l2(1e-2)))

  # compile mode
  opt='adam'
  model.compile(optimizer=opt, loss='mse', metrics=['mse',rmse,'mae','mape'])
  print(model.summary())

##Ejemplo de LSTM

Basado en el ejemplo completo que está explicado en https://adventuresinmachinelearning.com/keras-lstm-tutorial/

###Preparación y entrenamiento

In [0]:
if useLSTM:
  x_train_data = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
  y_train_data = y_train
  
  x_valid_data = x_valid.reshape((x_valid.shape[0], x_valid.shape[1], 1))
  y_valid_data = y_valid

  print('Entrenamiento:',x_train_data.shape, y_train_data.shape)
  print('Test:',x_valid_data.shape, y_valid_data.shape)

Entrenamiento: (20404, 99, 1) (20404, 1)
Test: (10051, 99, 1) (10051, 1)


Definimos los tamaños de nuestra red y algunas variables adicionales:

En nuestro caso, la entrada va a ser de *1* **x** *nº de datos tomados* **x** *nº de variables independientes*, siendo:

- batch_size, para cada instante tomamos los datos en paquetes.
- nº de datos tomados o num_steps, esto es, en el análisis secuencial cada registro energético completo con el que contamos, o en términos más coloquiales, cuántas filas del dataset se tienen en cuenta .
- nº de variables independientes o num_var, cada registro en los datos de entrada, o en términos más coloquiales, cada columna del dataset.

Además:

- hidden_size, número de unidades en cada célula del LSTM.

- A la salida, tenemos los tres valores a estimar.

In [0]:
if useLSTM:
  num_steps = split_steps - 1
  num_var = x_train_data.shape[2]
  output_size = y_train_data.shape[1]

###Modelo

A la capa LSTM se le pasa cada vez un instante, siendo el primero el t1, el segundo el t2, etc.

Respecto al ejemplo en la página, hemos:
- Quitado el embedding inicial, ya que no necesitamos codificar la entrada (ya son valores en sí mismos).
- Cambiado el hidden-layer-size. En la página dicen que se suele poner al tamaño de entrada de cada registro. Viene a ser el símil de unidades en una capa densa.
- Dejado una única capa LSTM.
- Eliminado el dropout.
- TimeDistributed usa una capa densa para cada step del entrenamiento. La quitamos también y dejamos la densa exclusivamente, ya que hace la salida muy grande.

Así:


In [0]:
if useLSTM:
  # define model
  model = Sequential()
  model.add(SimpleRNN(50,
                    activation='relu',
                    kernel_regularizer=l2(0.001),
                    return_sequences=True,
                    input_shape=(num_steps,num_var)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(SimpleRNN(50,
                      activation='relu',
                      kernel_regularizer=l2(0.001),
                      return_sequences=True))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(SimpleRNN(50,
                      activation='relu',
                      kernel_regularizer=l2(0.001)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  
  model.add(Dense(output_size,
                 activation='linear',
                 kernel_regularizer=l2(0.001)))

  # compile mode
  model.compile(optimizer='adam', loss='mse', metrics=['mse',rmse,'mae','mape'])
  model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_10 (SimpleRNN)    (None, 99, 50)            2600      
_________________________________________________________________
batch_normalization_7 (Batch (None, 99, 50)            200       
_________________________________________________________________
dropout_10 (Dropout)         (None, 99, 50)            0         
_________________________________________________________________
simple_rnn_11 (SimpleRNN)    (None, 99, 50)            5050      
_________________________________________________________________
batch_normalization_8 (Batch (None, 99, 50)            200       
_________________________________________________________________
dropout_11 (Dropout)         (None, 99, 50)            0         
__

##Entrenamiento

Entrenamos el modelo seleccionado:

In [0]:
# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1)

if useDense:
  print('DENSE')
  history_train = model.fit(x_train_data, y_train_data, 
                            epochs=5, 
                            batch_size=batch_size, 
                            shuffle=False,
                            verbose=1)

if useLSTM:
  print('LSTM')
  history_train = model.fit(x_train_data, y_train_data, 
                            epochs=50, 
                            batch_size=30, 
                            shuffle=False,
                            validation_split=0.3,
                            callbacks=[es],
                            verbose=1)

# summarize history for loss, MAPE in a different scale than the others
plt.plot(history_train.history['mean_squared_error'])
plt.plot(history_train.history['mean_absolute_error'])
plt.plot(history_train.history['rmse'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['MSE','MAE','RMSE'], loc='upper left')
plt.show()

plt.plot(history_train.history['mean_absolute_percentage_error'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['MAPE'], loc='upper left')
plt.show()

#Comments sobre las metrics: 

#Sobre MSE supongamos que tenemos la real de 0.6 y la pred de 0.3 como en los ejemplos, 
#Esto nos deja un error de 0.3, en el MSE 0.3^2 = 0.09, 
#es decir no concuerda con el MSE que nos sale de media 0.009
#Este MSE de 0.0092 significa que tenemos un error medio de 0.094 = Raiz(0.0092)
#Igualmente este error no es mucho, pero tampoco es bueno, significa que en el max min, 
#Entre 0 y 1 tenemos que el error medio es de 0.1, en real pueden ser como 20€, 
#que siendo un valor de 0 a 1 es bastante, no ajusta bien el precio!!

#El MAE lo confirma, este se acerca al 0.1, siendo del 0.07
#Este es el error medio, sin raíces si squared.

#Habría que ver como sale este valor para los valores reales, y que MSE Y RMSE da. 
#Si el MSE es de 0.0

#En consecuencia el MAPE, se nos va por las nubes, porque es el porcentaje del error frente al 
#valor verdaero, como hay un error muy grande, el porcentaje es grande y se nos va

#Tenemos que ver: 
#1.- Predicción con los valores sin scalar
#2.- Plot del prediccion total con el inversed value, para ver de verdad que predice.
#3.- MAD metric podría ser intersante

LSTM
Train on 14282 samples, validate on 6122 samples
Epoch 1/50
  360/14282 [..............................] - ETA: 26s - loss: 28.8782 - mean_squared_error: 28.7716 - rmse: 4.0012 - mean_absolute_error: 4.0012 - mean_absolute_percentage_error: 2298256.2782

##Test

Para el test:

In [0]:
if useDense:
  scores_test = model.evaluate(x_valid_data, y_valid_data, batch_size=batch_size, verbose=1)

if useLSTM:
  scores_test = model.evaluate(x_valid_data, y_valid_data, batch_size=batch_size, verbose=1)

  # summarize loss
  for element in range(len(scores_test)):
    print("%s: %.2f" % (model.metrics_names[element], scores_test[element]))

In [0]:
if useDense:
  x_valid_data_DF = pd.DataFrame(x_valid_data)
  
if useLSTM:
  x_valid_data_DF = pd.DataFrame(x_valid_data[:,:,0])
  
x_valid_data_DF.head()

In [0]:
if useDense:
  y_valid_data_DF = pd.DataFrame(y_valid_data)
  
if useLSTM:
  y_valid_data_DF = pd.DataFrame(y_valid_data)
  
y_valid_data_DF.head()

Valores que tratamos que sean iguales:

In [0]:
print(y_valid_data[0:9])

In [0]:
if useDense:
  print('DENSE')
  prediction = model.predict(x_valid_data[0:9,:], verbose=1)

if useLSTM:
  print('LSTM')
  prediction = model.predict(x_valid_data[0:9,:,:], verbose=1)

print(prediction)

In [0]:
x_train_data_temp = x_train_data
y_train_data_temp = y_train_data

for i_pred in range(1,25):
  prediction = model.predict(x_valid_data[(i_pred-1):i_pred,:], verbose=1)
  x_train_data_temp = np.append(x_train_data_temp, x_valid_data[(i_pred-1):i_pred, :], axis=0)
  y_train_data_temp = np.append(y_train_data_temp, prediction, axis=0)
  
 
  y_pred_val = scaler.inverse_transform(prediction)
  y_true_val = scaler.inverse_transform(y_valid_data[(i_pred-1):i_pred,:])
  print('True_val not sacled:',y_valid_data[(i_pred-1):i_pred,:])
  print('Pred_val not sacled:',prediction)
  print('True_val:',y_true_val)
  print('Pred_val:',y_pred_val)
  print('Perdida en €:',(y_true_val-y_pred_val))
  model.train_on_batch(x_train_data_temp, y_train_data_temp)
  
#Al Pasar un día debería hacer un fit de nuevo total, con los valores de verdad, pero estos predichos siguen siendo shit!  

In [0]:
 prediction = model.predict_on_batch(x_valid_data[0:9,:,:])
 print(prediction)


In [0]:
#Pesos
for layer in model.layers:
    print(layer.get_weights()[0])

In [0]:
#Biases
for layer in model.layers:
    print(layer.get_weights()[1])